In [2]:
import pandas as pd

In [116]:
just_relabel = "../../data/train_cleaning_label_5sj.csv"
just_relabel_output = "../../data/train_cleaning_label_5sj_aug800.csv"
with_denoise = "../../data/train_cleaning_label_5sj_denoise_add_by_llm_wo_error.csv"
with_denoise_output = "../../data/train_cleaning_label_5sj_denoise_add_by_llm_wo_error_aug800200.csv"

In [117]:
just_relabel_df = pd.read_csv(with_denoise)

print(just_relabel_df)

                       ID                                  text  target  \
0     ynat-v1_train_00000      정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보       4   
1     ynat-v1_train_00001           K찰.국DLwo 로L3한N% 회장 2 T0&}송=       3   
2     ynat-v1_train_00002                m 김정) 자주통일 새,?r열1나가야1보       2   
3     ynat-v1_train_00003         갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩       4   
4     ynat-v1_train_00004          pI美대선I앞두고 R2fr단 발] $비해 감시 강화       6   
...                   ...                                   ...     ...   
4366  ynat-v1_train_02787            1. 3인조 살인범 검거, 범인들은 전직 군인들       6   
4367  ynat-v1_train_02788                 남원 소식: 봄 학기 대학생 모집 안내       0   
4368  ynat-v1_train_02789    이낙연 총리, 4대 사회보험제도 완납 못해...남은 과제 많아       2   
4369  ynat-v1_train_02792      손흥민 선수 몸값 상승세 지속...유럽 내 최고 수준 도달       1   
4370  ynat-v1_train_02797  텔레그램 한달 무료 이용 등 아시아서 서비스 30분 단축...버전       4   

      is_noise  text_length  
0          NaN           32  
1          NaN           27  
2        

In [118]:
# Splitting into separate DataFrames by label
label_dfs = {label: just_relabel_df[just_relabel_df['target'] == label].copy() for label in range(7)}

In [119]:
def augment_data(df, label, num_samples=800):
    augmented_data = []
    for _ in range(num_samples):
        text1, text2 = df['text'].sample(2, replace=True).tolist()
        
        # 문장을 스페이스바 기준으로 나누기
        split_text1 = text1.split()
        split_text2 = text2.split()
        
        half_text1 = " ".join(split_text1[:len(split_text1) // 2])
        half_text2 = " ".join(split_text2[len(split_text2) // 2:])
        
        augmented_text = f"{half_text1} {half_text2}"
        augmented_data.append({"ID": f"aug_{label}_{_}", "text": augmented_text, "target": label})
    
    return pd.DataFrame(augmented_data)
def re_augment_data(df, label, num_samples=200):
    augmented_data = []
    for _ in range(num_samples):
        text1, text2 = df['text'].sample(2, replace=True).tolist()
        
        # 문장을 스페이스바 기준으로 나누기
        split_text1 = text1.split()
        split_text2 = text2.split()
        
        half_text1 = " ".join(split_text1[:len(split_text1) // 2])
        half_text2 = " ".join(split_text2[len(split_text2) // 2:])
        augmented_text = f"{half_text2} {half_text1}"
        augmented_data.append({"ID": f"aug_{label}_{_}", "text": augmented_text, "target": label})
    
    return pd.DataFrame(augmented_data)

In [120]:
# Augmenting each label DataFrame
augmented_dfs = [augment_data(df_label, label) for label, df_label in label_dfs.items()]
re_augmented_dfs = [re_augment_data(df_label, label) for label, df_label in label_dfs.items()]

# augmented_dfs와 re_augmented_dfs를 합침
all_augmented_dfs = augmented_dfs + re_augmented_dfs

# 원본 데이터프레임과 합친다
df_combined = pd.concat([just_relabel_df] + all_augmented_dfs, ignore_index=True)


In [121]:
df_combined


,ID,text,target,is_noise,text_length
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,NaN,32.0
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,NaN,27.0
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,NaN,22.0
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4,NaN,29.0
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,NaN,28.0
...,...,...,...,...,...
11366,aug_6_195,연 터키 최대 이슬람 사원 참르자 에마뉘엘 마크롱 프랑스 대통령,6,NaN,NaN
11367,aug_6_196,정권*진 요구종합 이란L우라늄 n|에 '성능,6,NaN,NaN
11368,aug_6_197,방송서 이스라엘과 군사협력 인정 호르무즈 호위연합 추진 美,6,NaN,NaN
11369,aug_6_198,강정호와 y별p T 스리랑카 말라リア 발병,6,NaN,NaN


In [122]:
df_combined.to_csv(with_denoise_output, index=False)